In [1]:
import tensorflow as tf
from tf_poly import TfPoly
import numpy as np

In [2]:
def runif():
    return tf.random.uniform([1], dtype=tf.float64)[0]

def rexp():
    return -tf.math.log(runif())

@tf.function
def gen_gaps(k: int, eta_x: tf.Tensor, eta_c: tf.Tensor, 
             theta=tf.constant(1e-4, dtype=tf.float64), 
             rho=tf.constant(1e-5, dtype=tf.float64)) -> tf.Tensor:
    '''Return k gaps sampled from genetic distribution with rate function eta.'''
    eta = TfPoly(x=eta_x, c=eta_c)
    R = eta.antiderivative()
    Rinv = R.inverse()
    x = Rinv(rexp())  # initialize x by sampling from prior
    pos = tf.constant(0., dtype=tf.float64)
    j = 0
    ta = tf.TensorArray(tf.float64, size=k + 2)
    while tf.less(j, k + 2):
        # x' satisfies R(x') - R(u*x) = Z => x' = Rinv(Z + R(u*x))
        u = runif()
        z = rexp()
        x = Rinv(z + R(u * x))  # segment height
        pos += rexp() / (x * (theta + rho))  # length to next event
        while runif() < (theta / (theta + rho)) and tf.less(j, k + 2):
            ta = ta.write(j, pos)
            j += 1
            pos += rexp() / (x * (theta + rho))  # length to next event
    ret = ta.stack()[1:]  # first obs suffers from inspection paradox?
    return ret[1:] - ret[:-1]

In [3]:
eta = TfPoly(x=[0., 1., 2., np.inf], c=[[2., 2., 3.]])
gen_gaps(2, eta.x, eta.c)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([7410.32566992, 5633.62401524])>

Check that $\mathbb{E}\nabla f(x) \approx \nabla \mathbb{E} f(x)$:

In [4]:
@tf.function
def grad_f(c1):
    c1 = tf.convert_to_tensor(c1)
    with tf.GradientTape() as tape:
        tape.watch(c1)
        eta = TfPoly(x=[0., 1., 2., np.inf], c=[[1., c1, 3.]])
        y = gen_gaps(2, eta.x, eta.c)[0]
    return tape.gradient(y, c1)

In [5]:
K = 1000
np.mean([grad_f(2.) for _ in range(K)])

703.3912

In [6]:
grads = []
c1 = tf.constant(2.0)
with tf.GradientTape() as tape:  # slow; graph grows with iteration count.
    tape.watch(c1)
    eta = TfPoly(x=[0., 1., 2., np.inf], c=[[1., c1, 3.]])
    _, ta = tf.while_loop(
        cond=lambda i, ta: i < K,
        body=lambda i, ta: (i + 1, ta.write(i, gen_gaps(2, eta.x, eta.c)[0])),
        loop_vars=(tf.constant(0), tf.TensorArray(size=K, dtype=tf.float64))
    )
    y = tf.reduce_mean(ta.stack())
print(tape.gradient(y, c1))

tf.Tensor(644.165, shape=(), dtype=float32)


It's close enough.

## How to use this function
Our parameter vector is `eta.c`. Goal is to use GAN to learn correct distribution of gaps generated for a particular choice of `c`. *Hope* is that the learned parameter vector matches original parameter vector. Example:

In [11]:
def generator(c):
    'Generate 10 gaps from distribution induced by c'
    eta = TfPoly(x=[0., 1., np.inf], c=[c])
    return gen_gaps(10, eta.x, eta.c)

training_data = [generator([5., 1.]) for _ in  range(1000)]

# use GAN to learn distribution; hope that embedding distribution concentrates on c=[5,1].